In [1]:
#Create a neural network graph and setup training steps for 
# a neural network to use and learn from the MNIST dataset.

# Import modules necessary for MNIST Experiment
import tensorflow as tf

#Import example
from tensorflow.examples.tutorials.mnist import input_data
#Setup data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784]) #Input values (N images by 784 pixels per image)
W = tf.Variable(tf.zeros([784, 10]))        #Weight values for input (784 pixels by 10 possible digits)
b = tf.Variable(tf.zeros([10]))             #Bias values for each digit
y = tf.nn.softmax(tf.matmul(x, W) + b)      #Output vector, Softmax(Wx + b)

# Define loss placeholder
y_ = tf.placeholder(tf.float32, [None, 10]) #How far is result from actual
# Create cross_entropy function based on loss and actual
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Instruct TensorFlow to attempt to minimize the cross_entropy using the Gradient Descent 
# Optimizer with a learning value of 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# importing the required packages
import numpy as np
from sklearn import manifold

# Initialize and save variables
init = tf.global_variables_initializer()
sess = tf.Session()     #Start new Session
sess.run(init)          #Initialize variables

#Number of steps to run
num_steps = 1000

#Number of samples for data collection
n_samples = 100
#Number of steps between sample collection
sample_gap = 10
#Define a way to save parameters for graph
graphs = np.zeros(shape=(num_steps // sample_gap + 1, #One dimension for each step in time
                         n_samples,                #One dimension each sample taken
                         2))                       #One dimension for vector data (2 from a 2d graph)
#Setup function for computing t-SNE for graphs
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
    
# Run training steps Num Steps times
for n in range(num_steps):
    #Before training the neural network, check if this is a sample step
    #Done before so it can include before training
    if n % sample_gap == 0:
        #Load n samples from the training dataset
        #Batch_x are the input images as a flattened 784 pixel vector
        #Batch_y is a vector of what number the image represents
        batch_x, batch_y = mnist.train.next_batch(n_samples)
        X = sess.run(y, feed_dict={x:batch_x})
        
        # (This is slow but precomputed) Computing t-SNE projection
        tsne_vector = tsne.fit_transform(X)
        graphs[n // sample_gap] = tsne_vector;
                  
    #Get a batch of 100 images
    #Batch_x are the input images as a flattened 784 pixel vector
    #Batch_y is a vector of what number the image represents
    batch_x, batch_y = mnist.train.next_batch(100)
    
    #Run the session with the given data (Do one training step)
    sess.run(train_step, feed_dict={x: batch_x, y_: batch_y})

#Final sample colleciton after training is complete
#Load n samples from the training dataset
#Batch_x are the input images as a flattened 784 pixel vector
#Batch_y is a vector of what number the image represents
batch_x, batch_y = mnist.train.next_batch(n_samples)
X = sess.run(y, feed_dict={x:batch_x})

# (This is slow but precomputed) Computing t-SNE projection
tsne_vector = tsne.fit_transform(X)
graphs[len(graph) - 1] = tsne_vector;

In [6]:
graphs

array([[[   0.        ,    0.        ],
        [   0.        ,    0.        ],
        [   0.        ,    0.        ],
        ..., 
        [   0.        ,    0.        ],
        [   0.        ,    0.        ],
        [   0.        ,    0.        ]],

       [[  78.56383175,   16.07070067],
        [-156.30930098,   82.60566259],
        [  -1.17504884,  -11.94092816],
        ..., 
        [  51.09054889,   45.10176147],
        [ -42.50160011,   14.63869253],
        [ -34.17398567,   42.00768936]],

       [[ -82.5500292 ,   74.37502643],
        [  75.7058456 ,   79.07511036],
        [   9.87229503,    8.8766247 ],
        ..., 
        [  93.92475286,   32.47346174],
        [ 125.98491289,    2.88938934],
        [ -11.54439463,  -87.79653522]],

       ..., 
       [[  -3.34964591,  -57.15510661],
        [ 107.1924515 ,   16.7840736 ],
        [ -43.03272037,  -27.82109739],
        ..., 
        [   8.70821454,  -65.18613558],
        [ -55.02370508,   36.92777754],
     